# Diário El Salvador - Preprocessing

In [15]:
from bs4 import BeautifulSoup
from datetime import date
from tqdm import tqdm

import numpy as np
import os
import pandas as pd
import requests

## Unifying the files and adding category column

In [3]:
csv_filenames = os.listdir('section_files')
dataframes = []
for file in csv_filenames:
    section_category = file.split('-')[3]
    df = pd.read_csv(f'section_files/{file}')
    df['category'] = section_category
    dataframes.append(df)

print(f'Number of dataframes: {len(dataframes)}')

Number of dataframes: 10


## Remove duplicates

In [4]:
df = pd.concat(dataframes, ignore_index=True)
print(f"Number of news: {len(df)}")
df.drop_duplicates(inplace=True)
print(f"Number of news without duplicates: {len(df)}")

Number of news: 16403
Number of news without duplicates: 16403


In [5]:
df.head()

,title,date,url,author,content,category
0,Así reaccionó Messi tras la victoria ante Bra...,22 de noviembre de 2023,https://diarioelsalvador.com/asi-reacciono-mes...,Jonathan Laguán,Pese a que el triunfo de 0-1 de Argentina sobr...,dedeportes
1,Rebeca Duarte le da a El Salvador la segunda ...,22 de noviembre de 2023,https://diarioelsalvador.com/rebeca-duarte-le-...,Raúl Recinos,La delegación salvadoreña que compite en los J...,dedeportes
2,Incorporan fondos para beneficio de sector ju...,hace 3 meses,https://diarioelsalvador.com/incorporan-fondos...,René Gómez,La comisión de Hacienda acordó emitir dictamen...,dedeportes
3,Inter Miami de Messi y Al Nassr de Cristiano ...,21 de noviembre de 2023,https://diarioelsalvador.com/inter-miami-de-me...,AFP,El Inter de Miami de Lionel Messi se enfrentar...,dedeportes
4,El Salvador presentará a la ONU informe de lo...,hace 1 año,https://diarioelsalvador.com/el-salvador-prese...,Mariela Quintanilla,El Consejo Nacional de Desarrollo Sostenible (...,dedeportes


## Date format

Some dates have a date format hard to convert for the YYY/MM/DD format. They have the 'hace' months or 'hace' year, for example, 'hace 3 meses' or 'hace 1 año'.

For this, we use the link to the news to get the complete date.

In [6]:
hace_date_format = df[df.date.str.strip().str.split(' ').str.get(0) == 'hace']
hace_date_format.head()

,title,date,url,author,content,category
2,Incorporan fondos para beneficio de sector ju...,hace 3 meses,https://diarioelsalvador.com/incorporan-fondos...,René Gómez,La comisión de Hacienda acordó emitir dictamen...,dedeportes
4,El Salvador presentará a la ONU informe de lo...,hace 1 año,https://diarioelsalvador.com/el-salvador-prese...,Mariela Quintanilla,El Consejo Nacional de Desarrollo Sostenible (...,dedeportes
6,«Tengo muchas ganas de empezar» con el Barcel...,hace 1 año,https://diarioelsalvador.com/tengo-muchas-gana...,AFP,"El centrocampista marfileño, Franck Kessié, as...",dedeportes
8,Lanzan programa de vivienda para beneficiar a...,hace 2 años,https://diarioelsalvador.com/lanzan-programa-d...,Magaly Abarca,"Más de 2,600 familias, es decir unos de 10,500...",dedeportes
10,Fotos: Una tormenta invernal «mayor» golpea e...,hace 2 años,https://diarioelsalvador.com/una-tormenta-inve...,AFP,Millones de estadounidenses se pusieron este d...,dedeportes


In [7]:
dates = []
for url in hace_date_format.url:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    datetime = soup.find("div", class_="jeg_meta_date")
    datetime = datetime.text
    dates.append(datetime)
    print(datetime, url)


 4 de septiembre de 2023 https://diarioelsalvador.com/incorporan-fondos-para-beneficio-de-sector-juventud/402416/
 27 de mayo de 2022 https://diarioelsalvador.com/el-salvador-presentara-a-la-onu-informe-de-los-objetivos-de-desarrollo-sostenible/231295/
 6 de julio de 2022 https://diarioelsalvador.com/tengo-muchas-ganas-de-empezar-con-el-barcelona-afirma-el-marfileno-franck-kessie/246636/
 27 de agosto de 2021 https://diarioelsalvador.com/lanzan-programa-de-vivienda-para-beneficiar-a-2600-familias-salvadorenas/125963/
 16 de enero de 2022 https://diarioelsalvador.com/una-tormenta-invernal-mayor-golpea-el-este-de-ee-uu/180178/
 30 de abril de 2021 https://diarioelsalvador.com/ministerio-de-salud-combate-el-zancudo-en-la-union-y-en-morazan/73366/
 18 de noviembre de 2021 https://diarioelsalvador.com/60-anos-de-prision-para-hombres-que-asesinaron-a-policia/159360/
 23 de marzo de 2021 https://diarioelsalvador.com/gobierno-despliega-paquetes-de-esperanza-a-todo-el-pais/55638/
 14 de noviemb

In [8]:
data_change_df = df[df.date.str.strip().str.split(' ').str.get(0) == 'hace'].copy()
data_change_df.head()

,title,date,url,author,content,category
2,Incorporan fondos para beneficio de sector ju...,hace 3 meses,https://diarioelsalvador.com/incorporan-fondos...,René Gómez,La comisión de Hacienda acordó emitir dictamen...,dedeportes
4,El Salvador presentará a la ONU informe de lo...,hace 1 año,https://diarioelsalvador.com/el-salvador-prese...,Mariela Quintanilla,El Consejo Nacional de Desarrollo Sostenible (...,dedeportes
6,«Tengo muchas ganas de empezar» con el Barcel...,hace 1 año,https://diarioelsalvador.com/tengo-muchas-gana...,AFP,"El centrocampista marfileño, Franck Kessié, as...",dedeportes
8,Lanzan programa de vivienda para beneficiar a...,hace 2 años,https://diarioelsalvador.com/lanzan-programa-d...,Magaly Abarca,"Más de 2,600 familias, es decir unos de 10,500...",dedeportes
10,Fotos: Una tormenta invernal «mayor» golpea e...,hace 2 años,https://diarioelsalvador.com/una-tormenta-inve...,AFP,Millones de estadounidenses se pusieron este d...,dedeportes


In [9]:
data_change_df.date = dates

In [10]:
data_change_df.head()

,title,date,url,author,content,category
2,Incorporan fondos para beneficio de sector ju...,4 de septiembre de 2023,https://diarioelsalvador.com/incorporan-fondos...,René Gómez,La comisión de Hacienda acordó emitir dictamen...,dedeportes
4,El Salvador presentará a la ONU informe de lo...,27 de mayo de 2022,https://diarioelsalvador.com/el-salvador-prese...,Mariela Quintanilla,El Consejo Nacional de Desarrollo Sostenible (...,dedeportes
6,«Tengo muchas ganas de empezar» con el Barcel...,6 de julio de 2022,https://diarioelsalvador.com/tengo-muchas-gana...,AFP,"El centrocampista marfileño, Franck Kessié, as...",dedeportes
8,Lanzan programa de vivienda para beneficiar a...,27 de agosto de 2021,https://diarioelsalvador.com/lanzan-programa-d...,Magaly Abarca,"Más de 2,600 familias, es decir unos de 10,500...",dedeportes
10,Fotos: Una tormenta invernal «mayor» golpea e...,16 de enero de 2022,https://diarioelsalvador.com/una-tormenta-inve...,AFP,Millones de estadounidenses se pusieron este d...,dedeportes


In [11]:
df.drop(hace_date_format.index, inplace=True)
df.head()

,title,date,url,author,content,category
0,Así reaccionó Messi tras la victoria ante Bra...,22 de noviembre de 2023,https://diarioelsalvador.com/asi-reacciono-mes...,Jonathan Laguán,Pese a que el triunfo de 0-1 de Argentina sobr...,dedeportes
1,Rebeca Duarte le da a El Salvador la segunda ...,22 de noviembre de 2023,https://diarioelsalvador.com/rebeca-duarte-le-...,Raúl Recinos,La delegación salvadoreña que compite en los J...,dedeportes
3,Inter Miami de Messi y Al Nassr de Cristiano ...,21 de noviembre de 2023,https://diarioelsalvador.com/inter-miami-de-me...,AFP,El Inter de Miami de Lionel Messi se enfrentar...,dedeportes
5,Scaloni prepara cambios en Argentina y advier...,20 de noviembre de 2023,https://diarioelsalvador.com/scaloni-prepara-c...,Graciela Guardado,"«El equipo lo tengo decidido, pero todavía no ...",dedeportes
7,Uriel Canjura campeón del XIV Torneo Internac...,19 de noviembre de 2023,https://diarioelsalvador.com/uriel-canjura-cam...,Rafael Cárcamo,"Uriel Canjura, medallista de bronce recienteme...",dedeportes


In [12]:
df = pd.concat([df, data_change_df])
df.head()

,title,date,url,author,content,category
0,Así reaccionó Messi tras la victoria ante Bra...,22 de noviembre de 2023,https://diarioelsalvador.com/asi-reacciono-mes...,Jonathan Laguán,Pese a que el triunfo de 0-1 de Argentina sobr...,dedeportes
1,Rebeca Duarte le da a El Salvador la segunda ...,22 de noviembre de 2023,https://diarioelsalvador.com/rebeca-duarte-le-...,Raúl Recinos,La delegación salvadoreña que compite en los J...,dedeportes
3,Inter Miami de Messi y Al Nassr de Cristiano ...,21 de noviembre de 2023,https://diarioelsalvador.com/inter-miami-de-me...,AFP,El Inter de Miami de Lionel Messi se enfrentar...,dedeportes
5,Scaloni prepara cambios en Argentina y advier...,20 de noviembre de 2023,https://diarioelsalvador.com/scaloni-prepara-c...,Graciela Guardado,"«El equipo lo tengo decidido, pero todavía no ...",dedeportes
7,Uriel Canjura campeón del XIV Torneo Internac...,19 de noviembre de 2023,https://diarioelsalvador.com/uriel-canjura-cam...,Rafael Cárcamo,"Uriel Canjura, medallista de bronce recienteme...",dedeportes


### YYY/MM/DD format

In [16]:
def date_format(date_string):
    months = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5,
        'junio': 6, 'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10,
        'noviembre': 11,'diciembre': 12}

    day, month, year = date_string.split('de')
    day = int(day.strip())
    month = months[month.strip()]
    year = int(year.strip())

    return date(year, month, day)

In [18]:
df.date = df.date.apply(date_format)
df.head()

,title,date,url,author,content,category
0,Así reaccionó Messi tras la victoria ante Bra...,2023-11-22,https://diarioelsalvador.com/asi-reacciono-mes...,Jonathan Laguán,Pese a que el triunfo de 0-1 de Argentina sobr...,dedeportes
1,Rebeca Duarte le da a El Salvador la segunda ...,2023-11-22,https://diarioelsalvador.com/rebeca-duarte-le-...,Raúl Recinos,La delegación salvadoreña que compite en los J...,dedeportes
3,Inter Miami de Messi y Al Nassr de Cristiano ...,2023-11-21,https://diarioelsalvador.com/inter-miami-de-me...,AFP,El Inter de Miami de Lionel Messi se enfrentar...,dedeportes
5,Scaloni prepara cambios en Argentina y advier...,2023-11-20,https://diarioelsalvador.com/scaloni-prepara-c...,Graciela Guardado,"«El equipo lo tengo decidido, pero todavía no ...",dedeportes
7,Uriel Canjura campeón del XIV Torneo Internac...,2023-11-19,https://diarioelsalvador.com/uriel-canjura-cam...,Rafael Cárcamo,"Uriel Canjura, medallista de bronce recienteme...",dedeportes


## Missing values

In [19]:
df.isnull().sum()

title         0
date          0
url           0
author        0
content     117
category      0
dtype: int64

In [20]:
df.dropna(inplace=True)
df.isnull().sum()

title       0
date        0
url         0
author      0
content     0
category    0
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16286 entries, 0 to 16402
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     16286 non-null  object
 1   date      16286 non-null  object
 2   url       16286 non-null  object
 3   author    16286 non-null  object
 4   content   16286 non-null  object
 5   category  16286 non-null  object
dtypes: object(6)
memory usage: 890.6+ KB


## Save file

In [22]:
df.to_csv('diario-el-salvador.csv', index=False)